In [17]:
%load_ext autoreload
%autoreload 2
import os
import typing as T
import pickle as pkl
import matplotlib.pyplot as plt
import sys
import cv2
import yaml
import requests
import torch
import scipy
import torch.nn as nn
import numpy as np
from xarray import DataArray
from tqdm.auto import tqdm
import io
from PIL import Image
from bioimageio_chatbot.image_processor import *
import numpy as np
from scipy.spatial.distance import cosine
from random import choice
import randomname

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Get the training data and pick representatives

In [7]:
cellpose_train_dir = "/Users/gkreder/Downloads/train"
cellpose_train_images = [os.path.join(cellpose_train_dir, x) for x in os.listdir(cellpose_train_dir) if x.endswith("_img.png")]
representative_images = get_representative_images(cellpose_train_images, verbose = True, threshold_distance=0.35)

  0%|          | 2/540 [00:00<00:42, 12.77it/s]

Embedding input images


100%|██████████| 540/540 [00:40<00:00, 13.22it/s]


Embedded database includes 540 vectors


Safety Counter Progress:   3%|▎         | 1607/54000 [04:04<2:13:04,  6.56it/s]


In [8]:
out_dir = "/Users/gkreder/Downloads/cellpose_train-set_representatives"
os.makedirs(out_dir, exist_ok=True)
for fname in representative_images:
    os.system(f"cp {fname} {out_dir}")

The 35 representatives images are currently stored in the gdrive folder "cellpose_train-set_representatives"

In [9]:
db_path = '../tmp/image_db/'
image_processor = ImageProcessor()
db = get_torch_db(db_path, image_processor)

2023-12-08 09:32:21.453 | INFO     | schema_agents.config:__init__:44 - Config loading done.
2023-12-08 09:32:21.454 | INFO     | schema_agents.config:__init__:55 - Set OPENAI_API_BASE in case of network issues


In [13]:
# entry[0] = result of get_torch_image. Results of get_model_torch_images()
# entry[1] = embedded image (in numpy array format). Result of image_process.embed_image
# entry[2] = rdf_dict (so need keys that match the nickname and axes at least)

In [18]:
cellpose_rep_dir = '/Users/gkreder/Downloads/cellpose_train-set_representatives/'
cellpose_image_paths = [os.path.join(cellpose_rep_dir, x) for x in os.listdir(cellpose_rep_dir) if not x.startswith(".")]
for fname_abs in cellpose_image_paths:
    fname = os.path.basename(fname_abs)
    break
# get_model_torch_images

In [24]:
model_zoo_names = [x[-1]['config']['bioimageio']['nickname'] for x in db]
rname = model_zoo_names[0]
while rname in model_zoo_names:
    rname = randomname.get_name(noun=("birds", "fish"))
rname

'straight-puffin'

In [37]:
cps = f"cellpose-{os.path.splitext(fname)[0].replace('_img', '')}"
nickname = f"{rname}_{cps}"
rdf_dict = {'config' : {'bioimageio' : {'nickname' : nickname}}}
rdf_dict['config']['bioimageio']['nickname']

'straight-puffin_cellpose-435'